In [1]:
import spacy
from collections import Counter
import random
import pandas as pd
ur = spacy.blank('ur')



In [2]:
punctuation = '''!()%\n٪-;۔،:\n\/'"\,“./؟_ء'''
exclude_list = ['٪', '!', '%', '`', '‘', '’', '"', ')', '(', '.', ':', "'", '"', '،', '؟']

li=[]
for i in punctuation:
    if i not in exclude_list:
        exclude_list.append(i)

In [3]:
files = ['ghalib.txt', 'iqbal.txt', 'faiz.txt']
all_lines = []
for file in files:
    with open(file, 'r', encoding="utf-8") as text:
        all_lines.extend(text.readlines())
#all_lines ; lines as list ->['بازیچۂ اطفال ہے دنیا مرے آگے\n',  'ہوتا ہے شب و روز تماشا مرے آگے\n',

In [4]:
def get_freq_dict(items):
    return dict(Counter(items))

def tokenize(sentences, backward):
    all_words = []
    for line in sentences:
        sent_tokens = []
        line = line.strip()
        if line:
            # sent_tokens.append('<s>')
            for i in reversed(list(ur(line))):
                sent_tokens.append(str(i))
            # sent_tokens.append('</s>')
        if backward:
            all_words = list(reversed(sent_tokens)) + all_words
        else:
            all_words = sent_tokens + all_words
    all_words = [i for i in all_words if i not in exclude_list]
    #print(len((all_words)))
    return all_words

In [5]:
def create_n_gram(words, n):
    combs_words = []
    next_prev_list = []
    for i in range(len(words) - n+1):
        c = [words[j] for j in range(i, i+n)]
        c_word = " ".join(c)
        combs_words.append(c_word)
        next_prev_list.append({"NextWord": c[-1], "PrevWords": " ".join(c[:n-1]), "Combined": c_word})

    n_gram = get_freq_dict(combs_words)
    return n_gram, next_prev_list

    
def generate_model(sentences, n, m_type='standard'):
    if m_type == 'backward':
        all_words = tokenize(sentences, backward=True)
    elif m_type == 'bidirectional':
        all_words = tokenize(sentences, backward=True) + tokenize(sentences, backward=False)
    else:
        all_words = tokenize(sentences, backward=False)
    #exclude_probs = ['<s>', '</s>', '‘', '’']
    unigram, _ = create_n_gram(all_words, 1)
    n_gram, next_prev_list = create_n_gram(all_words, n)
    
    return (unigram,n_gram,next_prev_list)
    

def generateDF(n_gram):
    df =  pd.DataFrame(n_gram.items(), columns=['ngram', 'freq'])
    return df

In [6]:
def genpoem(df,dfu):
    unigramList = dfu['ngram'].tolist()
    for stanza in range(3):

        for verse in range(4):
            singleVerse = []
            startingWord = random.choice(unigramList)

            for words in range(random.randint(3,7)):
                singleVerse.append(startingWord)
                singleVerse.append(' ')       
                next_word_list = df.loc[df["ngram"].str.startswith(startingWord)]
                next_word = pd.DataFrame(next_word_list['ngram'].str.split(n=2).str[1])
                startingWord = next_word['ngram'].iloc[0]

            print(*singleVerse, sep='')


        print('\n')

In [7]:
ui,ng,np=generate_model(all_lines,2,'standard')
dfu=generateDF(ui)
df=generateDF(ng)
df

,ngram,freq
0,وفا ہائے,6
1,ہائے نسخہ,6
2,نسخہ فیض,2
3,فیض احمد,2
4,احمد فیض,2
...,...,...
17233,و شب,1
17234,مرے دنیا,1
17235,دنیا ہے,1
17236,ہے اطفال,1


In [8]:
li=['standard','backward','bidirectional']
jk=2
for jk in range(2,4):
    for i in li:
        print("Ngram is",jk,"and model type is",i)
        ui,ng,np=generate_model(all_lines,jk,i)
        dfu=generateDF(ui)
        df=generateDF(ng)
        genpoem(df,dfu)

ngram is 2 and model type is standard
چھیڑے تذکرہ بدناں 
آئیں دیکھ دل خیر کی اس بری 
محرومی سے لحد کہ ہو روانہ بار 
چھیڑ تذکرہ بدناں 


دوڑے ہوے کئے فرہاد غم تجھے 
کج ہمیں نے اس بری بات کوئی 
پہنچا بہم سامان 
حوصلے کے روس وفا ہائے 


اپنی باقی نہیں ذرا پلے 
مغاں پیر تری سے لحد 
قط جو سارے آسماں میں 
سیر ہوائے سیماب رعشۂ کو 


ngram is 2 and model type is backward
گلِ و میرے پلے ذرا نہیں 
بغض کیا نہیں پڑتی لیکن اب 
بزم دہر سے لوٹ 
جستجو کرتا ہے بہ نام خدا اس 


گردشِ مدام سے لوٹ کے 
مکرر نہیں پڑتی لیکن 
جمع کرتے تھے ہم توف کی ہر 
ہوگی آہ سحر گئی مجھ بزم 


طریقت میں سمجھتی تھی 
ماسوا سمجھ تھی اس اب کوئی بات 
بدلہ غالب اسیری میں سمجھتی 
پوچھو تو کی ہر بات بری اسکی 


ngram is 2 and model type is bidirectional
اپنا بنا ہے بہ نام خدا اس 
خورشید عالم سارا گزرے 
عین فطرت خود 
برگِ سمن سے لوٹ 


کبریت سے لوٹ کے ملک الشعراء رسول 
مفسر مسجد پر ہوا محتسب 
خموشی میں سمجھتی تھی اس اب 
رہن ھیں تری گفتگو کریں 


مشاطۂ الفت چھپا کے 
لکھی ہے بہ نام 
گلہ کیا نہیں پڑتی لیکن اب کوئی 
جادۂ راہِ وفا میرے پ